In [1]:
import qiskit_metal as metal 

C:\Users\uday mathur\anaconda3\envs\Qiskit_metal\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.12.0-CAPI-1.18.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\users\uday mathur\qiskit-metal\qiskit_metal\__init__.py:30: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapel

In [2]:
from shapely.geometry import CAP_STYLE
from qiskit_metal import draw, Dict  # , QComponent
from qiskit_metal.qlibrary.core import QComponent 
from qiskit_metal.qlibrary.core import BaseQubit
import math 

class Configuration_B(  QComponent ):

    '''default_options = pos_x, pos_y,
                      pad_width_l,
                      pad_hight_l,
                      inductor_width_l,
                      pad_gap_l,
                      pad_gap_r,
                      inductor_width_r,
                      pad_width_r,
                      pad_hight_r,
                      connector_len,
                      con_dist_cen,
                      thickness_param
                      orientation = 
                      symmetric,
                      layer,
                      
                      chip)'''

      
    default_options = Dict( 
        pos_x = '0um',
        pos_y = '0um',
        pad_width_l = '100um',
        pad_height_l = '150um',
        inductor_width_l = '20um',
        pad_gap_l = '30um',
        pad_gap_r = '40um',
        inductor_width_r = '30um',
        pad_width_r = '90um',
        pad_height_r = '140um',
        connector_len = '180um',
        con_dist_cen = '60um',
        connector_width = '0.2um',
        orientation = '0',
        symmetric = False,
        pocket_width = '400um',
        pocket_height = '400um',
        epsilon = '0.2um',
        layer = '1',   
        chip = 'main')
    
    component_metadata = Dict(short_name='configuration_b',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    def make( self):
        p = self.p
        # chip = p.chip
        pos_x = p.pos_x
        pos_y = p.pos_y
        pad_width_l= p.pad_width_l
        pad_height_l = p.pad_height_l
        pad_gap_l = p.pad_gap_l
        pad_width_r= p.pad_width_r
        pad_height_r = p.pad_height_r
        pad_gap_r = p.pad_gap_r
        connector_len = p.connector_len
        connector_width = p.connector_width
        con_dist_cen = p.con_dist_cen

        # print("before",pad_width_l)
    
        if p.symmetric:
            pad_width_l = pad_width_r
            pad_height_l = pad_height_r
            pad_gap_l = pad_gap_r
            p.inductor_width_l = p.inductor_width_r
        # print("afteer",pad_width_l)
        
        pad_left = draw.rectangle( pad_width_l, pad_height_l)
        pad_right = draw.rectangle( pad_width_r, pad_height_r)
        
        pos_x_l = pos_x - ( connector_len / 2) - (pad_width_l/2)
        pos_y_l_top = pos_y + (pad_gap_l/2) + (pad_height_l/2)
        pos_y_l_bottom = pos_y - (pad_gap_l/2) - (pad_height_l/2)
        pos_x_r = pos_x + ( connector_len / 2) + (pad_width_r/2)
        pos_y_r_top = pos_y + (pad_gap_r/2) + (pad_height_r/2)
        pos_y_r_bottom = pos_y - (pad_gap_r/2) - (pad_height_r/2)
        
        
        pad_left_top = draw.translate( pad_left, pos_x_l, pos_y_l_top)
        pad_left_bottom = draw.translate( pad_left, pos_x_l, pos_y_l_bottom)
        pad_right_top = draw.translate( pad_right, pos_x_r, pos_y_r_top)
        pad_right_bottom = draw.translate( pad_right, pos_x_r, pos_y_r_bottom)

        if con_dist_cen >= min( (pad_gap_l/2 + pad_height_l), (pad_gap_r/2 + pad_height_r)):
            return False 
        connector1 = draw.rectangle(connector_len + 2*p.epsilon, connector_width)
        connector = draw.translate(connector1, pos_x , -con_dist_cen)
        rect_jj_l = draw.LineString([((pos_x - connector_len/2 - pad_width_l/2), pos_y + pad_gap_l/2),
                                      ((pos_x - connector_len/2 - pad_width_l/2), pos_y - pad_gap_l/2)])
        rect_jj_r =  draw.shapely.LineString( [( pos_x + connector_len/2 + pad_width_r/2, pos_y + pad_gap_r/2),
                                      ( pos_x + connector_len/2 + pad_width_r/2, pos_y - pad_gap_r/2)])

        down_geometry = draw.shapely.unary_union( [connector, pad_left_bottom, pad_right_bottom ])
        rect_pk = draw.rectangle(p.pocket_width, p.pocket_height)

        polys = [ pad_left_top,  pad_right_top, down_geometry]
        # polys = [ pad_left_top,  pad_right_top, pad_left_bottom, pad_right_bottom, rect_pk, connector]
        polys = draw.rotate( polys, p.orientation, origin = ( p.pos_x, p.pos_y))
        polys = draw.translate( polys, p.pos_x, p.pos_y)
        # [ pad_left_top,  pad_right_top, pad_left_bottom, pad_right_bottom, rect_pk, connector ] = polys
        [ pad_left_top,  pad_right_top, down_geometry] = polys

        # self.add_qgeometry('poly', dict(pad_left_top=pad_left_top , pad_left_bottom = pad_left_bottom))
        # self.add_qgeometry('poly', dict(pad_right_top=pad_right_top,pad_right_bottom = pad_right_bottom ))
        # self.add_qgeometry( 'poly', dict(  connector = connector))
        self.add_qgeometry('poly', dict(pad_left_top=pad_left_top ))
        self.add_qgeometry('poly', dict(pad_right_top=pad_right_top ))
        self.add_qgeometry('poly', dict( down_geometry = down_geometry  ))
        self.add_qgeometry('junction', dict(rect_jj_l=rect_jj_l), width=p.inductor_width_l)
        self.add_qgeometry('junction',dict(rect_jj_r=rect_jj_r), width=p.inductor_width_r)
        # self.add_qgeometry('poly', dict( down_geometry = down_geometry), chip = chip)
        self.add_qgeometry('poly', dict(rect_pk=rect_pk), subtract=True)


In [3]:
from qiskit_metal import designs, MetalGUI


In [4]:
design = designs.DesignPlanar()
gui = MetalGUI( design )
design.overwrite_enabled = True

In [102]:
q1 = Configuration_B( design, 'Q1' , options  = dict( connector_width = '5um', pocket_width = '900um', pocket_height = '900um', symmetric = True))
gui.rebuild()
gui.autoscale()

###### pad_left_bottom = draw.translate( pad_left, pos_x_l, pos_y_l_bottom)

In [103]:
from qiskit_metal.analyses.quantization import EPRanalysis 
eig_res = EPRanalysis( design, 'hfss')

In [104]:
hfss = eig_res.sim.renderer
hfss.start()

INFO 04:16PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:16PM [load_ansys_project]: 	Opened Ansys App
INFO 04:16PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 04:16PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/uday mathur/Documents/Ansoft/
	Project:   Project40
INFO 04:16PM [connect_design]: No active design found (or error getting active design).
INFO 04:16PM [connect]: 	 Connected to project "Project40". No design detected


True

In [105]:
hfss.activate_ansys_design( 'Two_qubit', 'eigenmode')

04:20PM 33s WARNING [activate_ansys_design]: The design_name=Two_qubit was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 04:20PM [connect_design]: 	Opened active design
	Design:    Two_qubit [Solution type: Eigenmode]
WARNING 04:20PM [connect_setup]: 	No design setup detected.
WARNING 04:20PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:20PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [106]:
hfss.render_design()

In [107]:
import pyEPR as epr
eig_res.sim.setup.max_passes = 10
eig_res.sim.setup.n_modes = 3
eig_res.sim.setup.vars.Lj = '11 nH'
eig_res.sim.setup.vars.Cj = '10 fF'
eig_res.sim.run()

INFO 04:20PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:20PM [analyze]: Analyzing setup Setup
04:22PM 04s INFO [get_f_convergence]: Saved convergences to C:\Users\uday mathur\Desktop\qiskit metal jupyter\qiskit_metal_practice_31\EPR_project_files\hfss_eig_f_convergence.csv


In [108]:
eig_res.sim.plot_convergences()

In [109]:
eig_res.sim.convergence_f

,re(Mode(1)) [g],re(Mode(2)) [g],re(Mode(3)) [g]
Pass [],,,
1,6.134880,8.546895,40.950840
2,6.972761,9.336193,41.582720
3,7.316063,9.771013,41.899061
4,7.540459,9.990948,42.175171
5,7.670177,10.169191,42.299084
6,7.757945,10.297896,42.413418
7,7.832961,10.377517,42.431744
8,7.863509,10.417683,42.492889
9,NaN,NaN,NaN


In [110]:
eig_res.sim.print_run_args()

This analysis object run with the following kwargs:
{}



In [111]:
# eig_res.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj_l'
# eig_res.setup.junctions.jj.line = 'JJ_Lj_Q1_joseph_'
# eig_res.setup

In [112]:
pinfo = epr.ProjectInfo( project_path = r'C:\Users\uday mathur\Desktop\qiskit metal jupyter\qiskit_metal_practice_31\EPR_project_files',
                        project_name = 'Configuration_B_CH2_5',
                        design_name = 'Two_qubit')

INFO 04:31PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:31PM [load_ansys_project]: 	File path to HFSS project found.
INFO 04:31PM [load_ansys_project]: 	Opened Ansys App
INFO 04:31PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 04:31PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/uday mathur/Desktop/qiskit metal jupyter/qiskit_metal_practice_31/EPR_project_files/
	Project:   Configuration_B_CH2_5
INFO 04:31PM [connect_design]: 	Opened active design
	Design:    Two_qubit [Solution type: Eigenmode]
INFO 04:31PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:31PM [connect]: 	Connected to project "Configuration_B_CH2_5" and design "Two_qubit" 😀 



In [113]:
eprh = epr.DistributedAnalysis( pinfo)

Design "Two_qubit" info:
	# eigenmodes    3
	# variations    1


In [114]:
pinfo.get_all_object_names()

['main',
 'sample_holder',
 'pad_left_top_Q1',
 'pad_right_top_Q1',
 'down_geometry_Q1',
 'JJ_rect_Lj_Q1_rect_jj_l',
 'JJ_rect_Lj_Q1_rect_jj_r',
 'ground_main_plane',
 'JJ_Lj_Q1_rect_jj_l_',
 'JJ_Lj_Q1_rect_jj_r_']

In [115]:
pinfo.junctions['j1'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj_l',
                             'line': 'JJ_Lj_Q1_rect_jj_l_',  'Cj_variable': 'Cj'}
pinfo.junctions['j2'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj_r',
                             'line': 'JJ_Lj_Q1_rect_jj_r_',  'Cj_variable': 'Cj'}
eig_res.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 3,
 'max_delta_f': 0.5,
 'max_passes': 10,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '11 nH', 'Cj': '10 fF'}}

In [116]:


eig_res.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 3,
 'max_delta_f': 0.5,
 'max_passes': 10,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '11 nH', 'Cj': '10 fF'}}

In [117]:
pinfo.get_all_object_names()

['main',
 'sample_holder',
 'pad_left_top_Q1',
 'pad_right_top_Q1',
 'down_geometry_Q1',
 'JJ_rect_Lj_Q1_rect_jj_l',
 'JJ_rect_Lj_Q1_rect_jj_r',
 'ground_main_plane',
 'JJ_Lj_Q1_rect_jj_l_',
 'JJ_Lj_Q1_rect_jj_r_']

In [118]:
pinfo.get_all_variables_names()

['Lj', 'Cj']

In [119]:
pinfo.junctions.items()

dict_items([('j1', {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj_l', 'line': 'JJ_Lj_Q1_rect_jj_l_', 'Cj_variable': 'Cj'}), ('j2', {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj_r', 'line': 'JJ_Lj_Q1_rect_jj_r_', 'Cj_variable': 'Cj'})])

In [120]:

eprh.n_modes = int(3)
eprh._list_variations = ("Cj='2fF' Lj='12.5nH'",)
eprh.variations = ['0', '1', '2']
eprh.update_ansys_info()
eprh.variations
eprh. get_variation_string()
eprh.get_variations()
eprh.n_variations
eprh.get_freqs_bare_pd('0')

,Freq. (GHz),Quality Factor
mode,,
0,7.863509,inf
1,10.417683,inf
2,42.492889,inf


In [121]:
ℰ_total  = eprh.calc_energy_electric(obj='AllObjects')
ℰ_total
# eprh.calc_p_junction_single(1,'0')
lv = eprh._get_lv()
eprh._parse_listvariations(lv)
eprh.get_ansys_variations()
eprh._list_variations  = ['0', '1', '2']
eprh = epr.DistributedAnalysis( pinfo)

Design "Two_qubit" info:
	# eigenmodes    3
	# variations    1


In [122]:
import warnings
warnings.filterwarnings("ignore")
eprh.do_EPR_analysis( append_analysis = True )


Variation 0  [1/1]

  Mode 0 at 7.86 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.9%  1.925e-24 2.148e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 94.90%
	j1              0.441617  (+)        0.0237171
		Energy fraction (Lj over Lj&Cj)= 94.90%
	j2              0.455928  (+)        0.0244857
		(U_tot_cap-U_tot_ind)/mean=7.13%

  Mode 1 at 10.42 GHz   [2/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.4%   1.64e-24 1.046e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_1j   sign s_1j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 91.39%
	j1              0.458091  (+)        0.0431794
		Energy fraction (Lj over Lj&Cj)= 91.39%
	j2             

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024382), None)

In [123]:
eprh.results

{'0': {'Pm':          j1            j2
  0  0.421309  4.349619e-01
  1  0.422199  4.090333e-01
  2  0.000011  7.920706e-09,
  'Pm_cap':          j1            j2
  0  0.022626  2.335966e-02
  1  0.039796  3.855524e-02
  2  0.000017  1.242164e-08,
  'Sm':    s_j1  s_j2
  0     1    -1
  1     1     1
  2    -1     1,
  'Om':                  0          1          2
  freq_GHz  7.863509  10.417683  42.492889,
  'sols':             U_H           U_E
  0  4.295718e-26  3.850495e-24
  1  2.091111e-26  3.280038e-24
  2  1.729815e-25  2.478336e-26,
  'Qm_coupling': Empty DataFrame
  Columns: []
  Index: [0, 1, 2],
  'Ljs': j1    1.100000e-08
  j2    1.100000e-08
  dtype: float64,
  'Cjs': j1    2.000000e-15
  j2    2.000000e-15
  dtype: float64,
  'Qs': 0    inf
  1    inf
  2    inf
  dtype: float64,
  'freqs_hfss_GHz': 0     7.863509
  1    10.417683
  2    42.492889
  dtype: float64,
  'hfss_variables': _Cj    10fF
  _Lj    11nH
  dtype: object,
  'modes': range(0, 3),
  'I_peak': 0    0  

In [124]:
eprh._nominal_variation

"Cj='10fF' Lj='11nH'"

In [125]:
eprh.calc_avg_current_J_surf_mag('0', 'JJ_rect_Lj_Q1_rect_jj_l', 'JJ_Lj_Q1_rect_jj_l_')

3.9847618630635654e-10

In [126]:
eprh.calc_energy_electric('0')

2.47833563910763e-26

In [127]:
ℰ_total  = eprh.calc_energy_electric(obj = 'AllObjects')


In [128]:
eprh.calc_energy_magnetic('0')

1.72981458339401e-25

In [129]:
eprh.get_freqs_bare_pd('0')

,Freq. (GHz),Quality Factor
mode,,
0,7.863509,inf
1,10.417683,inf
2,42.492889,inf


In [ ]:
# eprh.get_ansys_frequencies_all(0)

In [66]:
eprh._get_lv('0')

['Cj:=', '5fF', 'Lj:=', '11nH']

In [143]:
pinfo.design.solution_type
pinfo.setup_name

'Setup'

In [139]:
from qiskit_metal.analyses.simulation.scattering_impedance import ScatteringImpedanceSim
em1 = ScatteringImpedanceSim(design, "hfss")

In [141]:
hfss = em1.renderer
hfss.start()

INFO 04:41PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:41PM [load_ansys_project]: 	Opened Ansys App
INFO 04:41PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 04:41PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/uday mathur/Documents/Ansoft/
	Project:   Project40
INFO 04:41PM [connect_design]: No active design found (or error getting active design).
INFO 04:41PM [connect]: 	 Connected to project "Project40". No design detected


True

In [144]:
hfss.activate_ansys_design ( 'Two_Qubit', 'drivenmodal')

04:44PM 07s WARNING [activate_ansys_design]: The design_name=Two_Qubit was not in active project.  Designs in active project are: 
['Two_qubit'].  A new design will be added to the project.  


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024885), None)